In [8]:
import tensorflow as tf
from tensorflow.keras import layers

# 1. Dataset preparation (CIFAR-10 as an example)
def load_data():
    (train_images, _), (_, _) = tf.keras.datasets.cifar10.load_data()  # Load the CIFAR-10 dataset
    train_images = train_images.astype("float32") / 255.0  # Normalize images to [0, 1]
    return train_images

train_images = load_data()

# 2. Vision Transformer (ViT) model
class ViTImageGenerator(tf.keras.Model):
    def __init__(self, img_size=32, patch_size=4, embed_dim=64, num_heads=4, num_layers=6):
        super(ViTImageGenerator, self).__init__()

        # Parameters for the patch embedding
        self.patch_size = patch_size
        num_patches = (img_size // patch_size) ** 2  # Total number of patches

        # Dense layer to embed the patches
        self.patch_embed = layers.Dense(embed_dim)

        # Positional embedding to retain spatial information
        self.position_embed = tf.Variable(tf.random.normal([num_patches, embed_dim]))

        # Transformer layers for processing the patches
        self.encoder_layers = [
            layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim) for _ in range(num_layers)
        ]
        self.norm_layers = [layers.LayerNormalization() for _ in range(num_layers)]

        # Output layer to generate the image (scaled to [0,1] using sigmoid)
        self.fc = layers.Dense(img_size * img_size * 3, activation='sigmoid')

    def call(self, x):
        # Step 1: Extract patches from the input image
        patches = tf.image.extract_patches(
            images=x,
            sizes=[1, self.patch_size, self.patch_size, 1],  # Patch size
            strides=[1, self.patch_size, self.patch_size, 1],  # Move by patch size
            rates=[1, 1, 1, 1],
            padding='VALID'
        )

        # Get the batch size dynamically using tf.shape
        batch_size = tf.shape(x)[0]

        # Reshape patches to [batch, patches, channels], using batch_size
        patches = tf.reshape(patches, (batch_size, -1, patches.shape[-1]))

        # Step 2: Embed patches and add positional information
        x = self.patch_embed(patches)
        x += self.position_embed  # Adding position embedding

        # Step 3: Pass the embedded patches through transformer layers
        for enc_layer, norm_layer in zip(self.encoder_layers, self.norm_layers):
            x = norm_layer(x + enc_layer(x, x))  # Apply attention + residual connection

        # Step 4: Reconstruct the image from the processed patches
        x = self.fc(x)
        return tf.reshape(x, (-1, 32, 32, 3))  # Reshape the output back to image dimensions

# 3. Model training
def train_vit():
    # Instantiate the ViT model
    vit_model = ViTImageGenerator()
    vit_model.compile(optimizer='adam', loss='mse')  # Compile the model with Adam optimizer and MSE loss

    # Train the model (autoencoder style) to reconstruct the input image
    vit_model.fit(train_images, train_images, epochs=10, batch_size=64)

train_vit()  # Start training


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/mse/sub defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# 1. Dataset preparation (CIFAR-10 as an example)
def load_data():
    (train_images, _), (_, _) = tf.keras.datasets.cifar10.load_data()  # Load the CIFAR-10 dataset
    train_images = train_images.astype("float32") / 255.0  # Normalize images to [0, 1]
    return train_images

train_images = load_data()

# 2. Vision Transformer (ViT) model
class ViTImageGenerator(tf.keras.Model):
    def __init__(self, img_size=32, patch_size=4, embed_dim=64, num_heads=4, num_layers=6):
        super(ViTImageGenerator, self).__init__()

        self.img_size = img_size
        self.patch_size = patch_size
        num_patches = (img_size // patch_size) ** 2  # Total number of patches

        # Dense layer to embed the patches
        self.patch_embed = layers.Dense(embed_dim)

        # Positional embedding to retain spatial information
        self.position_embed = tf.Variable(tf.random.normal([num_patches, embed_dim]))

        # Transformer layers for processing the patches
        self.encoder_layers = [
            layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim) for _ in range(num_layers)
        ]
        self.norm_layers = [layers.LayerNormalization() for _ in range(num_layers)]

        # Output layer to generate the image (scaled to [0,1] using sigmoid)
        self.fc = layers.Dense(patch_size * patch_size * 3, activation='sigmoid') # Output per patch

    def call(self, x):
        # Step 1: Extract patches from the input image
        patches = tf.image.extract_patches(
            images=x,
            sizes=[1, self.patch_size, self.patch_size, 1],  # Patch size
            strides=[1, self.patch_size, self.patch_size, 1],  # Move by patch size
            rates=[1, 1, 1, 1],
            padding='VALID'
        )

        # Get the batch size dynamically using tf.shape
        batch_size = tf.shape(x)[0]

        # Reshape patches to [batch, patches, channels], using batch_size
        patches = tf.reshape(patches, (batch_size, -1, patches.shape[-1]))

        # Step 2: Embed patches and add positional information
        x = self.patch_embed(patches)
        x += self.position_embed  # Adding position embedding

        # Step 3: Pass the embedded patches through transformer layers
        for enc_layer, norm_layer in zip(self.encoder_layers, self.norm_layers):
            x = norm_layer(x + enc_layer(x, x))  # Apply attention + residual connection

        # Step 4: Reconstruct the image from the processed patches
        x = self.fc(x) # Apply fc layer to each patch

        # Reshape the output to image dimensions
        x = tf.reshape(x, (batch_size, self.img_size // self.patch_size, self.img_size // self.patch_size, self.patch_size * self.patch_size * 3))

        # Reshape to the original image size
        reconstructed = tf.nn.depth_to_space(x, self.patch_size)

        return reconstructed # Reshape the output back to image dimensions

# 3. Model training
def train_vit():
    # Instantiate the ViT model
    vit_model = ViTImageGenerator()
    vit_model.compile(optimizer='adam', loss='mse')  # Compile the model with Adam optimizer and MSE loss

    # Train the model (autoencoder style) to reconstruct the input image
    vit_model.fit(train_images, train_images, epochs=10, batch_size=32)

    return vit_model

# Train the model
vit_model = train_vit()

Epoch 1/10
 129/1563 ━━━━━━━━━━━━━━━━━━━━ 7:49 327ms/step - loss: 0.0682